In [ ]:
import json
from collections import defaultdict

def analyze_lime_explanations(file_path):
    """
    Analyzes LIME explanations from a JSON file.
    
    This function reads a JSON file containing LIME explanations, processes the feature influences,
    aggregates feature weights, and provides insights into the most important features for each label.
    
    Parameters:
        file_path (str): Path to the JSON file containing LIME explanations.
    """
    # Load the JSON file
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    # Data structures to store relevant metrics
    feature_influence = defaultdict(list)  # Stores all weightings per feature
    label_summary = defaultdict(list)  # Stores all weightings per label
    feature_mapping = {}  # Mapping from feature names to their top words

    # Iterate through all explanation instances
    for instance in data['explanations']:
        predicted_label = instance['predicted_label']  # Extract predicted label
        
        # Process each feature explanation
        for feature, weight in instance['explanation']:
            # Extract feature name (e.g., "feature_78")
            feature_name = feature.split(" ")[0]  
            feature_influence[feature_name].append(weight)  # Store weight for the feature

            # Store weight per label
            label_summary[predicted_label].append((feature_name, weight))
            
            # Extract words associated with the feature (within parentheses)
            words = feature.split("(")[1].split(")")[0].strip()  
            feature_mapping[feature_name] = words.split(", ")  # Store words as a list
    
    # Compute the average influence for each feature
    feature_avg_influence = {
        feature: sum(weights) / len(weights)
        for feature, weights in feature_influence.items()
    }

    # Compute the average influence for each label
    label_avg_influence = {}
    for label, features in label_summary.items():
        # Group features by label and compute their average influence
        feature_weights = defaultdict(list)
        for feature, weight in features:
            feature_weights[feature].append(weight)
        
        label_avg_influence[label] = {
            feature: sum(weights) / len(weights)
            for feature, weights in feature_weights.items()
        }

    # Sort features by their absolute average influence in descending order
    sorted_feature_influence = sorted(feature_avg_influence.items(), key=lambda x: abs(x[1]), reverse=True)

    # Output the top features with the highest influence
    print("Top features with the highest influence:")
    for feature, avg_weight in sorted_feature_influence[:10]:
        top_words = feature_mapping.get(feature, ["N/A"])  # Retrieve associated words
        print(f"{feature}: {avg_weight:.4f} -> ({', '.join(top_words)})")

    # Output the most important features per label
    for label, features in label_avg_influence.items():
        sorted_features_by_label = sorted(features.items(), key=lambda x: abs(x[1]), reverse=True)
        print(f"\nTop features for label {label}:")
        for feature, avg_weight in sorted_features_by_label[:5]:
            top_words = feature_mapping.get(feature, ["N/A"])  # Retrieve associated words
            print(f"{feature}: {avg_weight:.4f} -> ({', '.join(top_words)})")

# Define the path to the JSON file
file_path = "lime_label_explanations.json"

# Execute the analysis
analyze_lime_explanations(file_path)